In [1]:
#load libraries
import pandas as pd
import numpy as np
from pyzillow.pyzillow import ZillowWrapper, GetDeepSearchResults, GetUpdatedPropertyDetails
from datetime import datetime

In [2]:
#read data file
data = pd.read_csv("C:/Users/kalyaniK/Documents/Zillow data/SampleAddrFile.csv") #SecondMort
data.head()

,ProcessDate,PARENTACCOUNT,TYPE,Description,Ordinal,ID,FirstMortgageBalance_UserAmount1,Valuation_UserAmount2,SubjectPropertyStreet_UserChar1,SubjectPropertyExtraAddress_UserChar2,SubjectPropertyCity_UserChar3,SubjectPropertyCounty_UserChar4,SubjectPropertyState_UserChar5,SubjectPropertyZipcode_UserChar6
0,20180630,13862,31,PropertyAddressInfo,1,39,0.0,95900,5105 W HOMER AVE,NaN,TAMPA,NaN,FL,336297522
1,20180630,14159,31,PropertyAddressInfo,1,39,0.0,0,2913 WEST OSBORNE AVE,NaN,TAMPA,NaN,FL,33614
2,20180630,15180,31,PropertyAddressInfo,2,39,0.0,0,3202 N MACDILL AVE,NaN,TAMPA,NaN,FL,33607
3,20180630,17079,31,PropertyAddressInfo,1,39,0.0,314600,815 N VALRICO RD,NaN,VALRICO,NaN,FL,33594
4,20180630,17152,31,PropertyAddressInfo,0,39,0.0,0,705 S LOIS AVE,NaN,TAMPA,NaN,FL,336090000


In [3]:
#clean address fields
#convert to upper case 
data.SubjectPropertyStreet_UserChar1 = data['SubjectPropertyStreet_UserChar1'].astype(str).str.upper()
data.SubjectPropertyExtraAddress_UserChar2 = data['SubjectPropertyExtraAddress_UserChar2'].astype(str).str.upper()
data.SubjectPropertyCity_UserChar3 = data['SubjectPropertyCity_UserChar3'].astype(str).str.upper()
data.SubjectPropertyState_UserChar5 = data['SubjectPropertyState_UserChar5'].astype(str).str.upper()
data.SubjectPropertyZipcode_UserChar6 = data['SubjectPropertyZipcode_UserChar6'].astype(str).str.upper()
#remove leading and trailing white spaces
data.SubjectPropertyStreet_UserChar1 = data['SubjectPropertyStreet_UserChar1'].str.strip()
data.SubjectPropertyExtraAddress_UserChar2 = data['SubjectPropertyExtraAddress_UserChar2'].str.strip()
data.SubjectPropertyCity_UserChar3 = data['SubjectPropertyCity_UserChar3'].str.strip()
data.SubjectPropertyState_UserChar5 = data['SubjectPropertyState_UserChar5'].str.strip()
data.SubjectPropertyZipcode_UserChar6 = data['SubjectPropertyZipcode_UserChar6'].str.strip()

In [4]:
#clean zipcode- extract first 5 digits of zipcode only
data.SubjectPropertyZipcode_UserChar6 = data.SubjectPropertyZipcode_UserChar6.str[0:5]

In [5]:
#drop unrequired columns
data.drop(['Description', 'Ordinal','FirstMortgageBalance_UserAmount1', 'SubjectPropertyCounty_UserChar4'], 
                 axis=1, inplace=True)

In [6]:
#import zipcode data
zipcode_FL = pd.read_csv("C:/Users/kalyaniK/Documents/Zillow data/FL_zip_code.csv")
zipcode_FL.head()

,zip,city,state,latitude,longitude
0,32003,Fleming Island,FL,30.160000,-81.700000
1,32004,Ponte Vedra Beach,FL,29.937673,-81.420603
2,32006,Fleming Island,FL,30.110000,-81.720000
3,32007,Bostwick,FL,29.799631,-81.627324
4,32008,Branford,FL,29.928691,-82.910320


In [7]:
#convert zip to character
zipcode_FL.zip = zipcode_FL.zip.astype(str).str.upper()
zipcode_FL.zip = zipcode_FL.zip.str.strip()

In [8]:
#merge the two data frames -- when using multiple columns for join use [] to enclose ids separated by space
data_zip = pd.merge(data, zipcode_FL[['zip','city','state']],  
                           left_on = 'SubjectPropertyZipcode_UserChar6', right_on = 'zip', how = 'left') 
data_zip.head()

,ProcessDate,PARENTACCOUNT,TYPE,ID,Valuation_UserAmount2,SubjectPropertyStreet_UserChar1,SubjectPropertyExtraAddress_UserChar2,SubjectPropertyCity_UserChar3,SubjectPropertyState_UserChar5,SubjectPropertyZipcode_UserChar6,zip,city,state
0,20180630,13862,31,39,95900,5105 W HOMER AVE,NAN,TAMPA,FL,33629,33629,Tampa,FL
1,20180630,14159,31,39,0,2913 WEST OSBORNE AVE,NAN,TAMPA,FL,33614,33614,Tampa,FL
2,20180630,15180,31,39,0,3202 N MACDILL AVE,NAN,TAMPA,FL,33607,33607,Tampa,FL
3,20180630,17079,31,39,314600,815 N VALRICO RD,NAN,VALRICO,FL,33594,33594,Valrico,FL
4,20180630,17152,31,39,0,705 S LOIS AVE,NAN,TAMPA,FL,33609,33609,Tampa,FL


In [9]:
#merge the requried fields to form FullAddress (exclude zip)
second_mort_zip['FullAddress'] = second_mort_zip.apply(lambda x: '%s %s %s' % (x['SubjectPropertyStreet_UserChar1'],
                                                                              x['city'], x['state']), axis = 1)
second_mort_zip.FullAddress = second_mort_zip.FullAddress.str.upper()
second_mort_zip.head()

,ProcessDate,PARENTACCOUNT,TYPE,ID,Valuation_UserAmount2,SubjectPropertyStreet_UserChar1,SubjectPropertyExtraAddress_UserChar2,SubjectPropertyCity_UserChar3,SubjectPropertyState_UserChar5,SubjectPropertyZipcode_UserChar6,zip,city,state,FullAddress
0,20180630,13862,31,39,95900,5105 W HOMER AVE,NAN,TAMPA,FL,33629,33629,Tampa,FL,5105 W HOMER AVE TAMPA FL
1,20180630,14159,31,39,0,2913 WEST OSBORNE AVE,NAN,TAMPA,FL,33614,33614,Tampa,FL,2913 WEST OSBORNE AVE TAMPA FL
2,20180630,15180,31,39,0,3202 N MACDILL AVE,NAN,TAMPA,FL,33607,33607,Tampa,FL,3202 N MACDILL AVE TAMPA FL
3,20180630,17079,31,39,314600,815 N VALRICO RD,NAN,VALRICO,FL,33594,33594,Valrico,FL,815 N VALRICO RD VALRICO FL
4,20180630,17152,31,39,0,705 S LOIS AVE,NAN,TAMPA,FL,33609,33609,Tampa,FL,705 S LOIS AVE TAMPA FL


In [10]:
#drop unrequired columns
second_mort_zip.drop(['city', 'state','SubjectPropertyZipcode_UserChar6', 'SubjectPropertyState_UserChar5',
                     'SubjectPropertyCity_UserChar3', 'SubjectPropertyExtraAddress_UserChar2', 
                      'SubjectPropertyStreet_UserChar1'], axis=1, inplace=True)
second_mort_zip.head()

,ProcessDate,PARENTACCOUNT,TYPE,ID,Valuation_UserAmount2,zip,FullAddress
0,20180630,13862,31,39,95900,33629,5105 W HOMER AVE TAMPA FL
1,20180630,14159,31,39,0,33614,2913 WEST OSBORNE AVE TAMPA FL
2,20180630,15180,31,39,0,33607,3202 N MACDILL AVE TAMPA FL
3,20180630,17079,31,39,314600,33594,815 N VALRICO RD VALRICO FL
4,20180630,17152,31,39,0,33609,705 S LOIS AVE TAMPA FL


#ZILLOW ID AND LAST PRICE SOLD SEARCH USING pyzillow PACKAGE

In [32]:
#zillow API
zillow_api_key = 'X1-ZWz1gl2dn1skqz_7sxq3'
#zillow_api_key = 'X1-ZWz18g8p9imzuz_7galu'

In [33]:
#select 200 rows (from the data)
five_500 = second_mort_zip.iloc[1700:2200, :]
five_500.shape #check rows and cols

(500, 7)

In [34]:
five_500 = five_500.reset_index(drop=True) #reset index and drop the new col
five_500.head()

,ProcessDate,PARENTACCOUNT,TYPE,ID,Valuation_UserAmount2,zip,FullAddress
0,20180630,514869,31,39,153300,33592,10949 TOM FOLSOM RD THONOTOSASSA FL
1,20180630,514927,31,39,0,33705,2280 E BAY ISLE DR SE SAINT PETERSBURG FL
2,20180630,515056,31,39,0,33637,11201 DAVIS RD TAMPA FL
3,20180630,515072,31,39,335100,32308,2348 TOUR EIFFEL DR TALLAHASSEE FL
4,20180630,515353,31,39,210500,34453,2113 FOREST DR INVERNESS FL


In [35]:
#initialize values
zillowiddict=[]
resultindex =[]
errorlist=[]

In [36]:
data_set = five_500
#data_set['OrigIndex']=second_200.index
data_set.shape

(500, 7)

In [37]:
data_set.head(10)

,ProcessDate,PARENTACCOUNT,TYPE,ID,Valuation_UserAmount2,zip,FullAddress
0,20180630,514869,31,39,153300,33592,10949 TOM FOLSOM RD THONOTOSASSA FL
1,20180630,514927,31,39,0,33705,2280 E BAY ISLE DR SE SAINT PETERSBURG FL
2,20180630,515056,31,39,0,33637,11201 DAVIS RD TAMPA FL
3,20180630,515072,31,39,335100,32308,2348 TOUR EIFFEL DR TALLAHASSEE FL
4,20180630,515353,31,39,210500,34453,2113 FOREST DR INVERNESS FL
5,20180630,515775,31,39,219100,33615,8342 FOUNTAIN AVE TAMPA FL
6,20180630,515908,31,39,0,33610,3714 E SHAWDOWLAWN AV TAMPA FL
7,20180630,515981,31,21,0,33603,5101 N RIVER BLVD TAMPA FL
8,20180630,516401,31,39,728200,33609,4505 W SYLVAN RAMBLE ST TAMPA FL
9,20180630,516450,31,39,188600,33567,4605 HORTON RD PLANT CITY FL


In [38]:
for i in range(data_set.shape[0]):
    zillow_data = ZillowWrapper(zillow_api_key)
    try:
        deep_search_response = zillow_data.get_deep_search_results(data_set.FullAddress.loc[i],data_set.zip.loc[i])
        zillowiddict.append(GetDeepSearchResults(deep_search_response))
        resultindex.append(i)
    except:
        errorlist.append(i)

In [39]:
print(resultindex)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 30, 31, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 74, 75, 76, 77, 78, 79, 80, 81, 82, 84, 85, 86, 87, 88, 89, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 102, 103, 104, 105, 106, 107, 108, 109, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 184, 185, 186, 187, 188, 189, 190, 192, 193, 194, 195, 196, 197, 199, 200, 201, 202, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239,

In [40]:
len(errorlist)

34

In [41]:
print(errorlist) #indexes for which a match was not found on zillow

[19, 27, 28, 29, 32, 33, 72, 73, 83, 90, 101, 110, 142, 167, 168, 169, 183, 191, 198, 203, 242, 270, 288, 314, 332, 357, 394, 398, 400, 418, 423, 424, 434, 439]


In [42]:
zillowdf=pd.DataFrame()

In [43]:
#extract elements and store in pd dataframe
zillowdf['zillow_id']=pd.Series([i.zillow_id for i in zillowiddict])
zillowdf['home_type']=pd.Series([i.home_type for i in zillowiddict])
#zillowdf['home_detail_link']=pd.Series([i.home_detail_link for i in zillowiddict])
#zillowdf['graph_data_link']=pd.Series([i.graph_data_link for i in zillowiddict])
#zillowdf['map_this_home_link']=pd.Series([i.map_this_home_link for i in zillowiddict])
zillowdf['latitude']=pd.Series([i.latitude for i in zillowiddict])
zillowdf['longitude']=pd.Series([i.longitude for i in zillowiddict])
zillowdf['tax_year']=pd.Series([i.tax_year for i in zillowiddict])
zillowdf['tax_value']=pd.Series([i.tax_value for i in zillowiddict])
zillowdf['year_built']=pd.Series([i.year_built for i in zillowiddict])
zillowdf['property_size']=pd.Series([i.property_size for i in zillowiddict])
zillowdf['home_size']=pd.Series([i.home_size for i in zillowiddict])
zillowdf['area_unit']=pd.Series([i.area_unit for i in zillowiddict])
zillowdf['bathrooms']=pd.Series([i.bathrooms for i in zillowiddict])
zillowdf['bedrooms']=pd.Series([i.bedrooms for i in zillowiddict])
zillowdf['last_sold_date']=pd.Series([i.last_sold_date for i in zillowiddict])
zillowdf['last_sold_price']=pd.Series([i.last_sold_price for i in zillowiddict])
zillowdf['zestimate_amount']=pd.Series([i.zestimate_amount for i in zillowiddict])
zillowdf['zestimate_last_updated']=pd.Series([i.zestimate_last_updated for i in zillowiddict])
zillowdf['zestimate_value_change']=pd.Series([i.zestimate_value_change for i in zillowiddict])
zillowdf['zestimate_valuation_range_high']=pd.Series([i.zestimate_valuation_range_high for i in zillowiddict])
zillowdf['zestimate_valuationRange_low']=pd.Series([i.zestimate_valuationRange_low for i in zillowiddict])
#zillowdf['zestimate_percentile']=pd.Series([i.zestimate_percentile for i in zillowiddict])

In [44]:
zillowdf['DF_Index'] = resultindex

In [24]:
zillowdf.head(20)

,zillow_id,home_type,latitude,longitude,tax_year,tax_value,year_built,property_size,home_size,area_unit,bathrooms,bedrooms,last_sold_date,last_sold_price,zestimate_amount,zestimate_last_updated,zestimate_value_change,zestimate_valuation_range_high,zestimate_valuationRange_low,DF_Index
0,46329795,SingleFamily,28.249206,-82.36915,2017,330876.0,1992,81893,3249,SqFt,3.0,None,08/28/2000,64000,413653,08/14/2018,-18366,434336,392970,0
1,44993697,SingleFamily,27.891983,-82.291061,2017,194279.0,1970,17766,2001,SqFt,2.0,4,01/01/1970,29500,276802,08/14/2018,3810,290642,262962,1
2,45065332,SingleFamily,27.881242,-82.52597,2017,122959.0,1957,7140,1172,SqFt,2.0,3,None,None,220368,08/14/2018,7996,231386,209350,2
3,45116301,SingleFamily,28.039597,-82.391103,2017,136388.0,1954,8925,2144,SqFt,2.5,4,05/01/1977,36500,206051,08/14/2018,-1310,216354,183385,3
4,45116073,SingleFamily,28.038787,-82.394667,2017,128065.0,1958,7823,2386,SqFt,2.0,4,None,None,208057,08/14/2018,2704,218460,197654,4
5,43426944,SingleFamily,28.334662,-80.749127,2017,189240.0,1966,11326,2927,SqFt,2.0,3,04/01/1988,102500,281149,08/14/2018,-5279,303641,267092,5
6,44913235,SingleFamily,28.017529,-82.494052,2017,150032.0,1969,8025,2268,SqFt,2.0,4,04/01/1980,62500,242579,08/14/2018,4955,254708,230450,6
7,44913235,SingleFamily,28.017529,-82.494052,2017,150032.0,1969,8025,2268,SqFt,2.0,4,04/01/1980,62500,242579,08/14/2018,4955,254708,230450,7
8,44920800,SingleFamily,28.159552,-82.428573,2017,172859.0,1982,24640,1943,SqFt,2.0,3,04/01/1984,96000,263426,08/14/2018,-4513,276597,250255,8
9,45034827,SingleFamily,28.038774,-82.469431,2017,97775.0,1953,11700,1125,SqFt,1.0,2,None,None,171607,08/14/2018,7163,180187,163027,9


In [ ]:
#write as csv
#zillowdf.to_csv('C:/Users/kalyaniK/Documents/Zillow data/zillowdf.csv', sep=',')

In [45]:
#convert index to column
data_set.shape

(500, 7)

In [46]:
#assign index values to column
#data_set = data_set.reset_index(drop=True)
data_set['Index1'] = data_set.index
data_set.tail(10)

,ProcessDate,PARENTACCOUNT,TYPE,ID,Valuation_UserAmount2,zip,FullAddress,Index1
490,20180630,594648,31,39,0,33617,6615 JENNIFER DR TAMPA FL,490
491,20180630,594689,31,27,211100,33584,11820 DAVIS POOL ROAD SEFFNER FL,491
492,20180630,595041,31,39,0,34216,115 PELICAN DR ANNA MARIA FL,492
493,20180630,595041,31,40,880200,34216,115 PELICAN DR ANNA MARIA FL,493
494,20180630,595264,31,21,0,33610,1516 E LAKE AVE TAMPA FL,494
495,20180630,595397,31,39,280600,34655,6802 RIDGE TOP DR NEW PORT RICHEY FL,495
496,20180630,595421,31,40,232100,33618,12306 VERA AVE TAMPA FL,496
497,20180630,595637,31,40,216200,33594,228 SKYWOOD DR VALRICO FL,497
498,20180630,595702,31,41,289200,33624,15402 TIMBERLINE DR TAMPA FL,498
499,20180630,596031,31,39,132400,33619,1010 KIRKLAND DR TAMPA FL,499


In [47]:
#datatype of Index1
zillowdf['DF_Index'] = zillowdf['DF_Index'].astype(int)
data_set['Index1'] = data_set['Index1'].astype(int)

In [48]:
#merge the two dataframes by index id
result = pd.merge(data_set, zillowdf, left_on='Index1', right_on='DF_Index', how = 'left')

In [49]:
result.shape

(500, 28)

In [50]:
result

,ProcessDate,PARENTACCOUNT,TYPE,ID,Valuation_UserAmount2,zip,FullAddress,Index1,zillow_id,home_type,...,bathrooms,bedrooms,last_sold_date,last_sold_price,zestimate_amount,zestimate_last_updated,zestimate_value_change,zestimate_valuation_range_high,zestimate_valuationRange_low,DF_Index
0,20180630,514869,31,39,153300,33592,10949 TOM FOLSOM RD THONOTOSASSA FL,0,44964234,SingleFamily,...,2.0,3,None,None,182304,08/14/2018,13375,191419,173189,0.0
1,20180630,514927,31,39,0,33705,2280 E BAY ISLE DR SE SAINT PETERSBURG FL,1,47253650,SingleFamily,...,3.5,None,10/13/1993,8000,917132,08/14/2018,8914,962989,871275,1.0
2,20180630,515056,31,39,0,33637,11201 DAVIS RD TAMPA FL,2,44931154,SingleFamily,...,2.5,3,01/01/1972,5000,219837,08/14/2018,3027,233027,204448,2.0
3,20180630,515072,31,39,335100,32308,2348 TOUR EIFFEL DR TALLAHASSEE FL,3,45570715,SingleFamily,...,None,None,09/11/2000,168500,350430,08/14/2018,-1951,367951,332909,3.0
4,20180630,515353,31,39,210500,34453,2113 FOREST DR INVERNESS FL,4,43625595,SingleFamily,...,2.0,3,None,None,163438,08/14/2018,6355,184685,145460,4.0
5,20180630,515775,31,39,219100,33615,8342 FOUNTAIN AVE TAMPA FL,5,44872794,SingleFamily,...,2.0,3,None,None,244844,08/14/2018,3183,257086,232602,5.0
6,20180630,515908,31,39,0,33610,3714 E SHAWDOWLAWN AV TAMPA FL,6,45086278,SingleFamily,...,2.0,5,None,None,186913,08/14/2018,-1857,196259,177567,6.0
7,20180630,515981,31,21,0,33603,5101 N RIVER BLVD TAMPA FL,7,45093096,SingleFamily,...,3.5,3,12/09/2009,170000,392370,08/14/2018,3918,411988,372752,7.0
8,20180630,516401,31,39,728200,33609,4505 W SYLVAN RAMBLE ST TAMPA FL,8,45046871,SingleFamily,...,3.0,5,None,None,846038,08/14/2018,-6322,888340,803736,8.0
9,20180630,516450,31,39,188600,33567,4605 HORTON RD PLANT CITY FL,9,45030574,SingleFamily,...,1.0,2,None,None,156612,08/14/2018,8946,170707,139385,9.0


In [51]:
#write as csv
result.to_csv('C:/Users/kalyaniK/Documents/Zillow data/result.csv', sep=',')

In [52]:
#concatenate the csv files into one dataset and store it as a result
df1 = pd.read_csv('C:/Users/kalyaniK/Documents/Zillow data/result files/first200rows.csv', sep=',')
df2 = pd.read_csv('C:/Users/kalyaniK/Documents/Zillow data/result files/second300.csv', sep=',')
df3 = pd.read_csv('C:/Users/kalyaniK/Documents/Zillow data/result files/third200.csv', sep=',')
df4 = pd.read_csv('C:/Users/kalyaniK/Documents/Zillow data/result files/four500.csv', sep=',')
df5 = pd.read_csv('C:/Users/kalyaniK/Documents/Zillow data/result files/five500.csv', sep=',')
df6 = pd.read_csv('C:/Users/kalyaniK/Documents/Zillow data/result files/six500.csv', sep=',')

In [53]:
df1.head()

,ProcessDate,PARENTACCOUNT,TYPE,ID,Valuation_UserAmount2,zip,FullAddress,Index1,zillow_id,home_type,...,bathrooms,bedrooms,last_sold_date,last_sold_price,zestimate_amount,zestimate_last_updated,zestimate_value_change,zestimate_valuation_range_high,zestimate_valuationRange_low,DF_Index
0,20180630,13862,31,39,95900,33629.0,5105 W HOMER AVE TAMPA FL,0,45054743.0,SingleFamily,...,2.0,3.0,NaN,NaN,1621569.0,8/12/2018,23676.0,1702647.0,1540491.0,0.0
1,20180630,14159,31,39,0,33614.0,2913 WEST OSBORNE AVE TAMPA FL,1,45042211.0,SingleFamily,...,2.0,3.0,3/21/2016,205000.0,237556.0,8/12/2018,2208.0,249434.0,225678.0,1.0
2,20180630,15180,31,39,0,33607.0,3202 N MACDILL AVE TAMPA FL,2,45101634.0,SingleFamily,...,2.0,3.0,NaN,NaN,217576.0,8/12/2018,-4595.0,228455.0,195818.0,2.0
3,20180630,17079,31,39,314600,33594.0,815 N VALRICO RD VALRICO FL,3,45015824.0,SingleFamily,...,3.0,4.0,NaN,NaN,332281.0,8/12/2018,-32521.0,348895.0,315667.0,3.0
4,20180630,17152,31,39,0,33609.0,705 S LOIS AVE TAMPA FL,4,45047207.0,SingleFamily,...,4.0,5.0,6/4/2015,475000.0,927097.0,8/12/2018,20293.0,1038349.0,843658.0,4.0


In [57]:
complete_result = pd.concat([df1,df2,df3,df4,df5,df6], axis=0, ignore_index=False)

In [58]:
complete_result.shape

(2200, 29)

In [60]:
complete_result

,DF_Index,FullAddress,ID,Index1,PARENTACCOUNT,ProcessDate,TYPE,Unnamed: 0,Valuation_UserAmount2,area_unit,...,tax_value,tax_year,year_built,zestimate_amount,zestimate_last_updated,zestimate_valuationRange_low,zestimate_valuation_range_high,zestimate_value_change,zillow_id,zip
0,0.0,5105 W HOMER AVE TAMPA FL,39,0,13862,20180630,31,NaN,95900,SqFt,...,1112180.0,2017.0,1955.0,1621569.0,8/12/2018,1540491.0,1702647.0,23676.0,45054743.0,33629.0
1,1.0,2913 WEST OSBORNE AVE TAMPA FL,39,1,14159,20180630,31,NaN,0,SqFt,...,169778.0,2017.0,1963.0,237556.0,8/12/2018,225678.0,249434.0,2208.0,45042211.0,33614.0
2,2.0,3202 N MACDILL AVE TAMPA FL,39,2,15180,20180630,31,NaN,0,SqFt,...,146168.0,2017.0,1960.0,217576.0,8/12/2018,195818.0,228455.0,-4595.0,45101634.0,33607.0
3,3.0,815 N VALRICO RD VALRICO FL,39,3,17079,20180630,31,NaN,314600,SqFt,...,185613.0,2017.0,1955.0,332281.0,8/12/2018,315667.0,348895.0,-32521.0,45015824.0,33594.0
4,4.0,705 S LOIS AVE TAMPA FL,39,4,17152,20180630,31,NaN,0,SqFt,...,371835.0,2017.0,1951.0,927097.0,8/12/2018,843658.0,1038349.0,20293.0,45047207.0,33609.0
5,5.0,923 SYLVIA LANE TAMPA FL,39,5,18853,20180630,31,NaN,0,SqFt,...,159050.0,2017.0,1962.0,242209.0,8/12/2018,230099.0,254319.0,-6479.0,44895189.0,33613.0
6,6.0,2709 W OSBORNE AVE TAMPA FL,39,6,19174,20180630,31,NaN,0,SqFt,...,114225.0,2017.0,1966.0,185585.0,8/12/2018,176306.0,194864.0,1356.0,45043191.0,33614.0
7,7.0,9819 BROWNSTONE DR TAMPA FL,20,7,19182,20180630,31,NaN,191000,SqFt,...,128244.0,2017.0,1986.0,151713.0,8/12/2018,144127.0,175987.0,-10795.0,45034338.0,33612.0
8,8.0,8917 SOUTH MOBLEY RD ODESSA FL,22,8,23119,20180630,31,NaN,0,SqFt,...,345513.0,2017.0,1964.0,470034.0,8/12/2018,446532.0,493536.0,-1127.0,44852946.0,33556.0
9,9.0,8917 SOUTH MOBLEY RD ODESSA FL,39,9,23119,20180630,31,NaN,0,SqFt,...,345513.0,2017.0,1964.0,470034.0,8/12/2018,446532.0,493536.0,-1127.0,44852946.0,33556.0
